<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0976f62aa7a2bf154859a05e797dc9435f2e876af08b5ba90b2c11c2ead12748
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-14 15:03:19
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  3.00 K
Current:  1.45 C
-------------------
Today PnL: -28.92 K (-0.2%)
Current PnL: -20.45 L (-13.4%)
CY Booked + Current PnL: -6.93 L (-4.54%)
-------------------
Total profit:  1.83 L
Total loss:  -22.28 L
-------------------
Total Booked + Current PnL: 20.05 L (15.96%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.33%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.36 C
Est FTT PnL: 91.33 L (63.0%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.32%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,145.54,84.0,M-SC,1.89,161082.0,-13477.0,13515.0,0.67,-7.72,8.39,0.02,234.0,-1.00,1.11,40.11,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,101.92,51.0,M-SC,3.10,86977.0,-13800.0,13890.0,0.66,-13.69,15.97,0.09,253.0,-0.99,0.60,14.27,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-15.54,44.0,X-MC,3.46,88170.0,-2508.0,17414.0,-0.61,-2.77,19.75,16.44,63.0,-0.14,0.61,16.85,X40,ATH,PHARMA
43,ITC,452.00,-40.34,48.0,X-LC,0.92,199341.0,-797.0,21688.0,0.48,-0.40,10.88,10.44,5.0,-0.04,1.38,5.14,X40,NTT,FMCG
35,ICICIGI,2252.93,-16.25,53.0,X-MC,7.51,186195.0,11875.0,23330.0,-1.56,6.81,12.53,20.19,68.0,0.51,1.28,22.96,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,STARHEALTH,761.00,36.77,71.0,H-SC,15.11,281738.0,615.0,132248.0,3.40,0.22,46.94,47.26,171.0,0.00,1.94,50.95,XY24,NTT,INSURANCE
84,WHIRLPOOL,2270.00,-41.57,40.0,M-SC,9.30,114614.0,-184.0,94224.0,1.51,-0.16,82.21,81.92,236.0,-0.00,0.79,35.68,XR,NTT,DURABLES
7,ASIANTILES,137.00,7611.11,65.0,L-SC,16.47,84592.0,-9218.0,85836.0,1.39,-9.83,101.47,81.67,271.0,-0.11,0.58,63.82,XR,NTT,CERAMICS
82,VBL,671.64,-18.27,47.0,X-LC,6.07,292746.0,-23096.0,135746.0,1.29,-7.31,46.37,35.67,4.0,-0.17,2.02,5.53,X40N,ATH,FMCG
56,RAJESHEXPO,518.00,1719.44,59.0,L-SC,3.20,49637.0,-87540.0,87634.0,1.20,-63.82,176.55,0.07,268.0,-1.00,0.34,23.45,OX40N,NTT,JEWELLERY


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-49.70,36.0,L-SC,21.70,73728.0,-39821.0,79870.0,-5.02,-35.07,108.33,35.27,270.0,-0.50,0.51,80.45,XR,NTT,HOTELS
61,ROUTE,2227.21,-57.43,37.0,H-SC,23.09,69796.0,-63270.0,155150.0,-3.02,-47.55,222.29,69.05,140.0,-0.41,0.48,2.83,SR,ATH,IT
41,INFY,2275.00,-20.67,52.0,X-LC,4.21,319436.0,6548.0,165148.0,-2.73,2.09,51.70,54.87,2.0,0.04,2.20,10.55,X40,BTT,IT
65,SHALBY,327.00,1043.54,36.0,M-SC,16.71,155565.0,-25578.0,71047.0,-2.53,-14.12,45.67,25.10,250.0,-0.36,1.07,24.67,XY24,NTT,HEALTHCARE
76,TMPV,1065.00,-16.97,36.0,X-LC,0.06,255384.0,-21026.0,442197.0,-2.02,-7.61,173.15,152.37,3.0,-0.05,1.76,0.00,XY24,NTT,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-41.57,40.0,M-SC,9.30,114614.0,-184.0,94224.0,1.51,-0.16,82.21,81.92,236.0,-0.00,0.79,35.68,XR,NTT,DURABLES
69,STARHEALTH,761.00,36.77,71.0,H-SC,15.11,281738.0,615.0,132248.0,3.40,0.22,46.94,47.26,171.0,0.00,1.94,50.95,XY24,NTT,INSURANCE
86,ZYDUSLIFE,1286.17,-14.53,37.0,H-MC,5.75,208569.0,-1005.0,75669.0,-0.04,-0.48,36.28,35.63,119.0,-0.01,1.44,16.11,AR,ATH,PHARMA
85,WIPRO,420.00,-14.25,52.0,M-LC,5.65,151137.0,192.0,109257.0,-0.64,0.13,72.29,72.51,99.0,0.00,1.04,6.16,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-19.34,44.0,M-SC,10.37,146000.0,-1644.0,160965.0,-1.08,-1.11,110.25,107.91,216.0,-0.01,1.01,1.85,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,145.54,84.0,M-SC,1.89,161082.0,-13477.0,13515.0,0.67,-7.72,8.39,0.02,234.0,-1.00,1.11,40.11,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,101.92,51.0,M-SC,3.10,86977.0,-13800.0,13890.0,0.66,-13.69,15.97,0.09,253.0,-0.99,0.60,14.27,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-65.98,49.0,H-SC,2.46,223137.0,-46530.0,82873.0,-0.57,-17.25,37.14,13.47,158.0,-0.56,1.54,13.46,XY24,NTT,PAINTS
67,SIS,528.00,2019.17,47.0,H-SC,2.43,85527.0,-25505.0,49110.0,-0.15,-22.97,57.42,21.26,166.0,-0.52,0.59,15.56,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,STARHEALTH,761.00,36.77,71.0,H-SC,15.11,281738.0,615.0,132248.0,3.40,0.22,46.94,47.26,171.0,0.00,1.94,50.95,XY24,NTT,INSURANCE
62,SAMMAANCAP,326.00,17.65,52.0,M-SC,36.45,158490.0,9270.0,134907.0,-1.25,6.21,85.12,96.62,245.0,0.07,1.09,71.52,XY25,NTT,FINANCE
85,WIPRO,420.00,-14.25,52.0,M-LC,5.65,151137.0,192.0,109257.0,-0.64,0.13,72.29,72.51,99.0,0.00,1.04,6.16,XR,NTT,IT
38,INDIAMART,4810.62,-47.96,50.0,H-SC,4.60,129675.0,6339.0,125292.0,-1.23,5.14,96.62,106.72,139.0,0.05,0.89,29.72,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-47.96,50.0,H-SC,4.60,129675.0,6339.0,125292.0,-1.23,5.14,96.62,106.72,139.0,0.05,0.89,29.72,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-41.57,40.0,M-SC,9.30,114614.0,-184.0,94224.0,1.51,-0.16,82.21,81.92,236.0,-0.00,0.79,35.68,XR,NTT,DURABLES
85,WIPRO,420.00,-14.25,52.0,M-LC,5.65,151137.0,192.0,109257.0,-0.64,0.13,72.29,72.51,99.0,0.00,1.04,6.16,XR,NTT,IT
86,ZYDUSLIFE,1286.17,-14.53,37.0,H-MC,5.75,208569.0,-1005.0,75669.0,-0.04,-0.48,36.28,35.63,119.0,-0.01,1.44,16.11,AR,ATH,PHARMA
25,FINCABLES,1641.55,-19.34,44.0,M-SC,10.37,146000.0,-1644.0,160965.0,-1.08,-1.11,110.25,107.91,216.0,-0.01,1.01,1.85,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,11.91,26.0,X-SC,9.36,85324.0,-43346.0,88643.0,-1.76,-33.69,103.89,35.20,92.0,-0.49,0.59,0.00,X40,NTT,FOOTWEAR
27,GLAXO,3466.20,-20.38,27.0,X-MC,5.87,197728.0,-1556.0,79566.0,-0.92,-0.78,40.24,39.15,60.0,-0.02,1.36,27.99,X40,ATH,PHARMA
55,QUESS,986.00,-53.72,27.0,X-SC,37.31,46209.0,-18797.0,169726.0,-1.79,-28.92,367.30,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
46,JSWINFRA,342.00,-19.28,31.0,X-MC,5.38,189068.0,-11279.0,41784.0,-0.44,-5.63,22.10,15.23,66.0,-0.27,1.30,24.81,X40N,NTT,REALTY
23,DMART,5391.45,-16.93,32.0,X-LC,5.64,202550.0,-7857.0,67024.0,-0.28,-3.73,33.09,28.12,19.0,-0.12,1.40,19.01,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,1065.00,-16.97,36.0,X-LC,0.06,255384.0,-21026.0,442197.0,-2.02,-7.61,173.15,152.37,3.0,-0.05,1.76,0.00,XY24,NTT,AUTO
43,ITC,452.00,-40.34,48.0,X-LC,0.92,199341.0,-797.0,21688.0,0.48,-0.40,10.88,10.44,5.0,-0.04,1.38,5.14,X40,NTT,FMCG
34,HONAUT,58357.33,-25.10,36.0,X-SC,0.96,106530.0,-21408.0,68541.0,-0.01,-16.73,64.34,36.84,90.0,-0.31,0.73,9.06,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,-20.34,65.0,X-MC,1.00,184150.0,7412.0,47327.0,-0.25,4.19,25.70,30.97,75.0,0.16,1.27,17.28,X40,ATH,INSURANCE
16,CAMS,4762.00,-1.69,56.0,X-SC,1.61,259426.0,7570.0,54869.0,-0.34,3.01,21.15,24.79,86.0,0.14,1.79,26.84,X40N,NTT,MISC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,11.91,26.0,X-SC,9.36,85324.0,-43346.0,88643.0,-1.76,-33.69,103.89,35.20,92.0,-0.49,0.59,0.00,X40,NTT,FOOTWEAR
55,QUESS,986.00,-53.72,27.0,X-SC,37.31,46209.0,-18797.0,169726.0,-1.79,-28.92,367.30,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
76,TMPV,1065.00,-16.97,36.0,X-LC,0.06,255384.0,-21026.0,442197.0,-2.02,-7.61,173.15,152.37,3.0,-0.05,1.76,0.00,XY24,NTT,AUTO
53,PAGEIND,51769.93,-28.58,37.0,X-MC,7.68,158760.0,-4980.0,48327.0,0.27,-3.04,30.44,26.47,55.0,-0.10,1.10,0.59,X40,ATH,APPARELS
10,BAJAJHFL,181.50,-18.37,36.0,X-MC,6.54,176366.0,-24140.0,121287.0,-0.09,-12.04,68.77,48.45,64.0,-0.20,1.22,1.70,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,986.00,-53.72,27.0,X-SC,37.31,46209.0,-18797.0,169726.0,-1.79,-28.92,367.30,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
59,RELAXO,1176.00,-41.83,37.0,X-SC,3.46,79355.0,-65805.0,141736.0,-1.73,-45.33,178.61,52.31,91.0,-0.46,0.55,6.22,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,11.91,26.0,X-SC,9.36,85324.0,-43346.0,88643.0,-1.76,-33.69,103.89,35.20,92.0,-0.49,0.59,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-15.54,44.0,X-MC,3.46,88170.0,-2508.0,17414.0,-0.61,-2.77,19.75,16.44,63.0,-0.14,0.61,16.85,X40,ATH,PHARMA
34,HONAUT,58357.33,-25.10,36.0,X-SC,0.96,106530.0,-21408.0,68541.0,-0.01,-16.73,64.34,36.84,90.0,-0.31,0.73,9.06,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-26.54,58.0,X-LC,8.64,291334.0,-54622.0,121311.0,-0.21,-15.79,41.64,19.28,1.0,-0.45,2.01,7.70,X40,ATH,IT
41,INFY,2275.00,-20.67,52.0,X-LC,4.21,319436.0,6548.0,165148.0,-2.73,2.09,51.70,54.87,2.0,0.04,2.20,10.55,X40,BTT,IT
76,TMPV,1065.00,-16.97,36.0,X-LC,0.06,255384.0,-21026.0,442197.0,-2.02,-7.61,173.15,152.37,3.0,-0.05,1.76,0.00,XY24,NTT,AUTO
82,VBL,671.64,-18.27,47.0,X-LC,6.07,292746.0,-23096.0,135746.0,1.29,-7.31,46.37,35.67,4.0,-0.17,2.02,5.53,X40N,ATH,FMCG
43,ITC,452.00,-40.34,48.0,X-LC,0.92,199341.0,-797.0,21688.0,0.48,-0.40,10.88,10.44,5.0,-0.04,1.38,5.14,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-49.70,36.0,L-SC,21.70,73728.0,-39821.0,79870.0,-5.02,-35.07,108.33,35.27,270.0,-0.50,0.51,80.45,XR,NTT,HOTELS
7,ASIANTILES,137.00,7611.11,65.0,L-SC,16.47,84592.0,-9218.0,85836.0,1.39,-9.83,101.47,81.67,271.0,-0.11,0.58,63.82,XR,NTT,CERAMICS
51,MASFIN,398.61,-18.70,52.0,H-SC,7.48,92580.0,-5400.0,27006.0,0.77,-5.51,29.17,22.05,164.0,-0.20,0.64,34.57,XR,ATH,FINANCE
70,SURYODAY,240.00,51.32,40.0,H-SC,13.37,139332.0,-39739.0,105474.0,0.26,-22.19,75.70,36.71,167.0,-0.38,0.96,38.25,XR,NTT,BANKS
80,VAIBHAVGBL,521.00,72.45,61.0,H-SC,8.47,149319.0,-33456.0,144526.0,-0.21,-18.30,96.79,60.77,159.0,-0.23,1.03,34.47,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,0.49,86.0,X-LC,8.84,255596.0,3828.0,49381.0,0.87,1.52,19.32,21.13,31.0,0.08,1.76,37.48,X40,ATH,PAINTS
39,INDIGOPNTS,1408.00,145.54,84.0,M-SC,1.89,161082.0,-13477.0,13515.0,0.67,-7.72,8.39,0.02,234.0,-1.00,1.11,40.11,OX40N,NTT,PAINTS
80,VAIBHAVGBL,521.00,72.45,61.0,H-SC,8.47,149319.0,-33456.0,144526.0,-0.21,-18.30,96.79,60.77,159.0,-0.23,1.03,34.47,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7611.11,65.0,L-SC,16.47,84592.0,-9218.0,85836.0,1.39,-9.83,101.47,81.67,271.0,-0.11,0.58,63.82,XR,NTT,CERAMICS
5,ANGELONE,3033.00,59.94,74.0,X-SC,7.21,224696.0,33690.0,24020.0,0.98,17.64,10.69,30.21,97.0,1.40,1.55,42.20,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.61
1,25,44.55
2,50,75.73


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    47.48
MC    27.76
LC    24.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.44
X40      20.54
X40N     12.26
XY25     10.06
XR        9.54
AR        8.98
OX40N     7.54
X200      1.80
SR        1.00
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.15
X-MC    21.41
X-LC    19.84
M-SC    13.33
X-SC     8.57
H-MC     4.66
H-LC     2.86
L-SC     1.43
M-MC     1.36
M-LC     1.04
L-LC     0.99
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.71,-4.02,37.66
IT,12.48,-16.17,78.28
FINANCE,11.91,-9.49,59.11
MISC,7.51,-19.15,79.25
PAINTS,6.03,-5.65,21.40
ELECTRICAL,5.56,-12.12,51.87
INSURANCE,4.49,3.05,31.12
AUTO,3.43,-15.25,125.76
PHARMA,3.41,-1.03,34.92


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3328485.0,20
XR,1270128.0,13
AR,1269150.0,10
X40,993375.0,14
OX40N,698823.0,10
X40N,691998.0,9
XY25,476907.0,7
SR,274661.0,2
MH,66223.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3499400.0,25
M-SC,1417449.0,16
X-LC,1190829.0,11
X-MC,1142966.0,15
X-SC,834404.0,8
H-MC,403117.0,3
L-SC,253340.0,3
H-LC,143085.0,2
M-LC,109257.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1177249.0      6
           AR         844625.0      5
           XR         797848.0      7
M-SC       XY24       741432.0      6
X-LC       XY24       504761.0      2
X-MC       X40        415336.0      7
X-LC       X40        409115.0      5
X-SC       XY24       376314.0      2
X-MC       XY24       344582.0      3
H-SC       OX40N      338794.0      4
X-SC       X40N       289166.0      4
H-SC       SR         274661.0      2
M-SC       OX40N      272395.0      5
H-MC       AR         218970.0      2
X-LC       X40N       202770.0      2
X-MC       X40N       200062.0      3
H-MC       XY24       184147.0      1
X-MC       XY25       182986.0      2
X-SC       X40        168924.0      2
L-SC       XR         165706.0      2
M-SC       XR         143079.0      2
           XY25       134907.0      1
           AR         125636.0      2
M-LC       XR         109257.0      1
L-SC       OX40N       87634.0      1
H-LC       AR          79919.0      1
X-LC       XY25        74183.0      2
H-SC       MH          66223.0      1
H-LC       X200        63166.0      1
L-MC       XR          54238.0      1
M-MC       XY25        48541.0      1
L-LC       XY25        36290.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
